# 참가 대회 정보

- 대회명 : 은행 이탈 데이터 세트를 사용한 이진 분류
- 링크 : https://www.kaggle.com/competitions/playground-series-s4e1
- 평가지표 : ROC curve

# 모듈가져오기

In [ ]:
import numpy as np
import pandas as pd

# 파일불러오기

In [ ]:
bank_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/9.파이널프로젝트/data/분류_은행이탈데이터/test.csv')
bank_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/9.파이널프로젝트/data/분류_은행이탈데이터/train.csv')
bank_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/9.파이널프로젝트/data/분류_은행이탈데이터/sample_submission.csv')

# 데이터 전처리

## shape, 결측치 확인

- test 데이터

In [ ]:
bank_test.shape, bank_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               110023 non-null  int64  
 1   CustomerId       110023 non-null  int64  
 2   Surname          110023 non-null  object 
 3   CreditScore      110023 non-null  int64  
 4   Geography        110023 non-null  object 
 5   Gender           110023 non-null  object 
 6   Age              110023 non-null  float64
 7   Tenure           110023 non-null  int64  
 8   Balance          110023 non-null  float64
 9   NumOfProducts    110023 non-null  int64  
 10  HasCrCard        110023 non-null  float64
 11  IsActiveMember   110023 non-null  float64
 12  EstimatedSalary  110023 non-null  float64
dtypes: float64(5), int64(5), object(3)
memory usage: 10.9+ MB


((110023, 13), None)

In [ ]:
bank_test.head(5)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


- train 데이터

In [ ]:
bank_train.shape, bank_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


((165034, 14), None)

In [ ]:
bank_train.head(5)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


- sample_submission 데이터

In [ ]:
bank_sub.shape, bank_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      110023 non-null  int64  
 1   Exited  110023 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.7 MB


((110023, 2), None)

In [ ]:
bank_sub.head()

,id,Exited
0,165034,0.5
1,165035,0.5
2,165036,0.5
3,165037,0.5
4,165038,0.5


## 필요없는 피처 제거

- 피처 정보
```
    - id : 인덱스
    - CustomerId : 각 고객의 고유 식별자
    - Surname : 고객의 성
    - CreditScore : 고객의 신용 점수를 나타내는 값
    - Geography : 고객이 거주하는 국가 (프랑스, 스페인, 독일)
    - Gender : 고객의 성별
    - Age : 고객의 연령
    - Tenure : 고객 은행 이용 기간(년)
    - Balance : 고객의 계좌 잔고
    - NumOfProducts : 고객이 사용하는 은행 상품 수 (예:저축 계좌, 신용 카드)
    - HasCrCard : 고객의 신용카드 보유 여부 (1=예, 0=아니오)
    - IsActiveMember : 고객 활동 여부 (1=예, 0=아니오)
    - EstimatedSalary : 고객 추정 급여
    - Exited : 고객 이탈 여부 (1=예, 0=아니오)
```


- 가설 검증에 필요한 피처만 남기는 작업 진행
- 가설 설정
```
    - 고객 이탈 현상은 고객의 연령대와 상관관계가 있을 것이다.
    - 고객 이탈 현상과 신용 점수, 계좌 잔고, 급여는 밀접한 연관이 있을 것이다.
    - 은행 이용 기간이 길수록 이탈율이 낮을 것이다.
```

- 제거할 피처
    - Surname
    - Geography
    - Gender




In [ ]:
bank_test = bank_test.drop(['Surname','Geography','Gender'], axis=1)
bank_train = bank_train.drop(['Surname','Geography','Gender'], axis=1)

In [ ]:
bank_test.head(2)

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,586,23.0,2,0.0,2,0.0,1.0,160976.75
1,165035,15782418,683,46.0,2,0.0,1,1.0,0.0,72549.27


In [ ]:
bank_train.head(2)

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,668,33.0,3,0.0,2,1.0,0.0,181449.97,0
1,1,15749177,627,33.0,1,0.0,2,1.0,1.0,49503.50,0


## 데이터 분포 확인

In [ ]:
bank_test.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,110023.000000,1.100230e+05,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000,110023.000000
mean,220045.000000,1.569210e+07,656.530789,38.122205,4.996637,55333.611354,1.553321,0.753043,0.495233,112315.147765
std,31761.048671,7.168499e+04,80.315415,8.861550,2.806148,62788.519675,0.544714,0.431244,0.499980,50277.048244
min,165034.000000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000
25%,192539.500000,1.563286e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74440.325000
50%,220045.000000,1.569018e+07,660.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117832.230000
75%,247550.500000,1.575693e+07,710.000000,42.000000,7.000000,120145.605000,2.000000,1.000000,1.000000,154631.350000
max,275056.000000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000


In [ ]:
bank_train.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


# 파일 저장

In [ ]:
bank_test.to_csv('/content/drive/MyDrive/Colab Notebooks/9.파이널프로젝트/data/분류_은행이탈데이터/bank_test.csv', index=False)
bank_train.to_csv('/content/drive/MyDrive/Colab Notebooks/9.파이널프로젝트/data/분류_은행이탈데이터/bank_train.csv', index=False)
# 데이터 전처리 결과 'sample_submission.csv'파일은 변경 사항 없음